# 1- Preprocessing

## Open the file

In [2]:
import pandas as pd

file_path = 'hw_data.csv'
df = pd.read_csv("hw_data.csv")
df

C:\Users\laptop\AppData\Local\Temp\ipykernel_9480\1625294764.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Unnamed: 0,Age,Gender,Education Level,Income,Marital Status,Employment Status,Product Category,Satisfaction Level,Debt Status
0,0,500,Female,NaN,31158.0,NaN,NaN,NaN,9,1
1,1,73,Female,College,54465.0,NaN,0.0,E,5,1
2,2,68,Male,High School,37427.0,married,0.0,D,1,1
3,3,61,Female,College,52210.0,widowed,0.0,G,8,1
4,4,34,Female,College,59325.0,separated,1.0,A,5,0
...,...,...,...,...,...,...,...,...,...,...
105,56,85,Female,University,39063.0,divorced,NaN,F,8,0
106,0,500,Female,NaN,31158.0,NaN,NaN,NaN,9,0
107,73,39,Female,College,55750.0,married,1.0,F,5,1
108,28,80,Female,High School,32142.0,separated,NaN,C,7,1


## Examine file

In [3]:
# Unnamed: 0: An index-like column.
# Age: There is a noticeable outlier (500).
# Education Level: There are some missing values.
# Income: There are some missing values.
# Marital Status: There are some missing values.
# Employment Status: Represented numerically (float), there are some missing values.
# Product Category: Represented by letters, There are some missing values.
# Satisfaction Level: Represented numerically (integer). 
# Debt Status:Represented numerically (integer).

## Define missing values

In [4]:
missing_values = df.isnull().sum()

missing_values[missing_values > 0]

Education Level      17
Income               12
Marital Status       19
Employment Status    35
Product Category     13
dtype: int64

## Decide about missing values (fill or drop)

In [5]:
# For categorical columns, fill with their mode
for column in ['Education Level', 'Marital Status', 'Product Category']:
    mode_value = df[column].mode()[0]
    df[column] = df[column].fillna(mode_value)

# For the 'Income' column, fill with its median
median_income = df['Income'].median()
df['Income'] = df['Income'].fillna(median_income)

# For the 'Employment Status', fill its missing values with its mode
mode_employment_status = df['Employment Status'].mode()[0]
df['Employment Status'] = df['Employment Status'].fillna(mode_employment_status)

# Check if there are any missing values left
missing_values_after = df.isnull().sum()

# Check for any remaining missing values
missing_values_after[missing_values_after > 0]

df


,Unnamed: 0,Age,Gender,Education Level,Income,Marital Status,Employment Status,Product Category,Satisfaction Level,Debt Status
0,0,500,Female,University,31158.0,separated,1.0,G,9,1
1,1,73,Female,College,54465.0,separated,0.0,E,5,1
2,2,68,Male,High School,37427.0,married,0.0,D,1,1
3,3,61,Female,College,52210.0,widowed,0.0,G,8,1
4,4,34,Female,College,59325.0,separated,1.0,A,5,0
...,...,...,...,...,...,...,...,...,...,...
105,56,85,Female,University,39063.0,divorced,1.0,F,8,0
106,0,500,Female,University,31158.0,separated,1.0,G,9,0
107,73,39,Female,College,55750.0,married,1.0,F,5,1
108,28,80,Female,High School,32142.0,separated,1.0,C,7,1


## Why did you fill missing values or if you have dropped, why did you do so? 

In [6]:
# Categorical columns ("Education Level", "Marital Status", "Product Category", and "Employment Status") were filled with their modes.
# Numerical column "Income" column's missing values were filled with its median.
# There are no missing values left in the dataset. 

## Find outliers if there are

In [7]:
# To calculate IQR and determine outliers
def find_outliers_iqr(data, column_name):
    Q1 = data[column_name].quantile(0.25)
    Q3 = data[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column_name] < lower_bound) | (data[column_name] > upper_bound)]

# Identifying outliers in 'Age'
age_outliers = df[df['Age'] > 100]
print(age_outliers)  

# Identifying outliers in 'Income'
income_outliers = find_outliers_iqr(df, 'Income')
print(income_outliers)

(len(age_outliers), len(income_outliers))

     Unnamed: 0  Age  Gender Education Level       Income Marital Status  \
0             0  500  Female      University      31158.0      separated   
10           10  500  Female         College      31654.0  never married   
20           20  500  Female      University      64165.0      separated   
30           30  500  Female     High School      18442.0      separated   
40           40  500  Female         College      45140.0        widowed   
50           50  500    Male      University  575772080.0        married   
60           60  500  Female         College      34060.0      separated   
70           70  500    Male     High School      21593.0  never married   
80           80  500    Male      University      60010.0        married   
90           90  500  Female      University      31759.0      separated   
106           0  500  Female      University      31158.0      separated   
109          30  500  Female     High School      18442.0      separated   

     Employ

(12, 13)

12 entries are considered outliers, with ages above 100.
13 entries are identified as outliers based on the Interquartile Range (IQR) method.

## Decide what will you do with outliers

In [8]:
# For "Age", replacing any value above 100 with the median age of the dataset to avoid distorting the data with extreme values.
# For "Income", since these outliers might represent valid extreme income values, we need to carefully consider whether to adjust these values or leave them as is. 
# Google says that A common approach is to cap these outliers at a certain percentile (e.g., the 95th percentile) to reduce their impact on the analysis without completely removing them.

median_age = df['Age'].median()
df.loc[df['Age'] > 100, 'Age'] = median_age

income_95th_percentile = df['Income'].quantile(0.95)
df.loc[df['Income'] > income_95th_percentile, 'Income'] = income_95th_percentile

df

,Unnamed: 0,Age,Gender,Education Level,Income,Marital Status,Employment Status,Product Category,Satisfaction Level,Debt Status
0,0,54,Female,University,31158.0,separated,1.0,G,9,1
1,1,73,Female,College,54465.0,separated,0.0,E,5,1
2,2,68,Male,High School,37427.0,married,0.0,D,1,1
3,3,61,Female,College,52210.0,widowed,0.0,G,8,1
4,4,34,Female,College,59325.0,separated,1.0,A,5,0
...,...,...,...,...,...,...,...,...,...,...
105,56,85,Female,University,39063.0,divorced,1.0,F,8,0
106,0,54,Female,University,31158.0,separated,1.0,G,9,0
107,73,39,Female,College,55750.0,married,1.0,F,5,1
108,28,80,Female,High School,32142.0,separated,1.0,C,7,1


## Find duplicates, how many are there? Decide what will you do about that?

In [9]:
# Identifying duplicate rows in the dataset
duplicates = df[df.duplicated()]

# Count of duplicate rows
duplicate_count = len(duplicates)
duplicate_count


4

In [10]:
# Remove duplicate rows
df_cleaned = df.drop_duplicates()

# Checking the count of duplicates again
duplicate_count_after = len(df_cleaned[df_cleaned.duplicated()])
duplicate_count_after


0

## Explain why did you do that?

In [11]:
# To ensure that each row represents a unique observation.

# 2- Categorization

## Categorize satisfaction in 3 level and label them as 'Low', 'Moderate' and 'High'. Add a new column named 'Satisfaction Category' and store them there

#### ||| Low Satisfaction : (1-3) ||| Moderate Satisfaction : (4-7) ||| High Satisfaction : (8-10) |||

In [12]:
def categorize_satisfaction(level):
    if 1 <= level <= 3:
        return 'Low'
    elif 4 <= level <= 7:
        return 'Moderate'
    else:
        return 'High'

# Apply the function to the 'Satisfaction Level' column to create the new 'Satisfaction Category' column
df['Satisfaction Category'] = df['Satisfaction Level'].apply(categorize_satisfaction)

df

,Unnamed: 0,Age,Gender,Education Level,Income,Marital Status,Employment Status,Product Category,Satisfaction Level,Debt Status,Satisfaction Category
0,0,54,Female,University,31158.0,separated,1.0,G,9,1,High
1,1,73,Female,College,54465.0,separated,0.0,E,5,1,Moderate
2,2,68,Male,High School,37427.0,married,0.0,D,1,1,Low
3,3,61,Female,College,52210.0,widowed,0.0,G,8,1,High
4,4,34,Female,College,59325.0,separated,1.0,A,5,0,Moderate
...,...,...,...,...,...,...,...,...,...,...,...
105,56,85,Female,University,39063.0,divorced,1.0,F,8,0,High
106,0,54,Female,University,31158.0,separated,1.0,G,9,0,High
107,73,39,Female,College,55750.0,married,1.0,F,5,1,Moderate
108,28,80,Female,High School,32142.0,separated,1.0,C,7,1,Moderate


## Categorize income in 3 level and label them as 'Low', 'Moderate' and 'High'. Add a new column named 'Income Category' and store them there

In [13]:
# Define quantiles for categorization: 0.33 and 0.66 will roughly divide the data into three equal parts
quantiles = df['Income'].quantile([0.33, 0.66])

# Define a function to categorize income
def categorize_income(income):
    if income <= quantiles.iloc[0]:
        return 'Low'
    elif income <= quantiles.iloc[1]:
        return 'Moderate'
    else:
        return 'High'

# Apply the function to the 'Income' column to create the new 'Income Category' column
df['Income Category'] = df['Income'].apply(categorize_income)

df

,Unnamed: 0,Age,Gender,Education Level,Income,Marital Status,Employment Status,Product Category,Satisfaction Level,Debt Status,Satisfaction Category,Income Category
0,0,54,Female,University,31158.0,separated,1.0,G,9,1,High,Low
1,1,73,Female,College,54465.0,separated,0.0,E,5,1,Moderate,High
2,2,68,Male,High School,37427.0,married,0.0,D,1,1,Low,Moderate
3,3,61,Female,College,52210.0,widowed,0.0,G,8,1,High,High
4,4,34,Female,College,59325.0,separated,1.0,A,5,0,Moderate,High
...,...,...,...,...,...,...,...,...,...,...,...,...
105,56,85,Female,University,39063.0,divorced,1.0,F,8,0,High,Moderate
106,0,54,Female,University,31158.0,separated,1.0,G,9,0,High,Low
107,73,39,Female,College,55750.0,married,1.0,F,5,1,Moderate,High
108,28,80,Female,High School,32142.0,separated,1.0,C,7,1,Moderate,Low


# 3- Analyse

## Comparing using pivot tables

### Is there a relation between marital status and debt status?

In [15]:
marital_debt_relation = pd.crosstab(index=df_cleaned['Marital Status'], columns=df_cleaned['Debt Status'], normalize='index')
marital_debt_relation

Debt Status,0,1
Marital Status,,
divorced,0.416667,0.583333
married,0.521739,0.478261
never married,0.400000,0.600000
separated,0.560976,0.439024
widowed,0.650000,0.350000


### Is there a relation between income category and debt status?

In [18]:
income_debt_relation = pd.crosstab(index=df_cleaned['Income'], columns=df_cleaned['Debt Status'], normalize='index')
income_debt_relation

Debt Status,0,1
Income,,
6.740000e+03,0.0,1.0
7.010000e+03,1.0,0.0
1.084500e+04,0.0,1.0
1.410600e+04,0.0,1.0
1.664700e+04,0.0,1.0
...,...,...
1.951440e+08,1.0,0.0
2.864215e+08,1.0,0.0
3.002459e+08,0.0,1.0


### Is there a relation between education level and satisfaction category?

In [22]:
df_cleaned = df.copy()

df_cleaned['Satisfaction Category'] = df_cleaned['Satisfaction Level'].apply(lambda x: 'High Satisfaction' if x > df_cleaned['Satisfaction Level'].median() else 'Low Satisfaction')

education_satisfaction_relation = pd.crosstab(index=df_cleaned['Education Level'], columns=df_cleaned['Satisfaction Category'], normalize='index')

print(education_satisfaction_relation)

Satisfaction Category  High Satisfaction  Low Satisfaction
Education Level                                           
College                         0.421053          0.578947
Elementary                      0.187500          0.812500
High School                     0.357143          0.642857
University                      0.382979          0.617021


### Is there a relation between employment status and satisfaction category?

In [20]:
employment_satisfaction_relation = pd.crosstab(index=df_cleaned['Employment Status'], columns=df_cleaned['Satisfaction Category'], normalize='index')
employment_satisfaction_relation

Satisfaction Category,High Satisfaction,Low Satisfaction
Employment Status,,
0.0,0.290323,0.709677
1.0,0.386667,0.613333


# 4- General Conclusion

Divorced: Approximately 58.33% have a debt status of 1 (assumed to indicate debt), while 41.67% have a debt status of 0 (no debt).
Married: About 47.83% have a debt status of 1, and 52.17% have a debt status of 0.
Never Married: 60% have a debt status of 1, while 40% have a debt status of 0.
Separated: Around 43.9% have a debt status of 1, and 56.1% have a debt status of 0.
Widowed: 35% have a debt status of 1, and 65% have a debt status of 0.

---

### Checklist

- [x]  start homework;
- [ ]  file open;
- [ ]  file examined;
- [ ]  missing values defined;
- [ ]  missing values are filled;
- [ ]  explanation for the possible causes of missing values;
- [ ]  an explanation of how the blanks are filled;
- [ ]  an explanation of how the blanks are filled;
- [ ]  finding outliers;
- [ ]  handling outliers;
- [ ]  duplicates showed;
- [ ]  duplicates deleted;
- [ ]  an explanation of which method is used to find and remove duplicates;
- [ ]  data is categorized;
- [ ]  an answer to the question " Is there a relation betweenmarital status and debt status?";
- [ ]  an answer to the question " Is there a relation between income category and debt status?";
- [ ]  an answer to the question " Is there a relation between education level and satisfaction category?";
- [ ]  an answer to the question " Is there a relation between employment status and satisfaction category?"
- [ ]  conclusions are present on each stage;
- [ ]  a general conclusion is made.

---